In [2]:
import numpy as np

In [15]:
from renjuRL_DK.renju_game.brains.BoardManager import B_SHAPE, PLAYERS

In [1]:
import keras

Using TensorFlow backend.


In [4]:
from keras import Model

In [2]:
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout

In [5]:
data_input = Input(shape=(15, 15, 3), name='BoardStatus')

Conv1 = Conv2D(64, (2, 2), padding='same', activation='relu')(data_input)
BN1 = BatchNormalization()(Conv1)
Conv2 = Conv2D(64, (3, 3), padding='same', activation='relu')(BN1)
BN2 = BatchNormalization()(Conv2)
Conv3 = Conv2D(64, (3, 3), padding='same', activation='relu')(BN2)
BN3 = BatchNormalization()(Conv3)
Pool1 = MaxPool2D()(BN3)

FLT = Flatten()(Pool1)
FC1 = Dense(500, activation='sigmoid', input_shape=(7 * 7 * 32,))(FLT)
DO1 = Dropout(0.5)(FC1)
FC2 = Dense(500, activation='sigmoid', input_shape=(500,))(DO1)
DO2 = Dropout(0.5)(FC2)
FC3 = Dense(15 * 15 + 1, activation='softmax', input_shape=(500,))(DO2)

policyNN = Model(inputs=data_input, outputs=FC3)

In [6]:
policyNN.summary()
policyNN.compile('adam',
                 loss='categorical_crossentropy',
                 metrics=[
                     keras.metrics.categorical_accuracy
                 ])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BoardStatus (InputLayer)     (None, 15, 15, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 64)        832       
_________________________________________________________________
batch_normalization_4 (Batch (None, 15, 15, 64)        256       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 15, 64)        256       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 64)        36928     
_________________________________________________________________
batch_normalization_6 (Batch (None, 15, 15, 64)        256       
__________

In [8]:
policyNN2 = keras.models.load_model('v1')

Instructions for updating:
Use tf.cast instead.


In [17]:
X_RANGE = range(15, 0, -1)
X_RANGE = [str(x) for x in X_RANGE]
X_MAPPING = dict(zip(X_RANGE, range(len(X_RANGE))))

Y_RANGE = list("abcdefghijklmno")
Y_MAPPING = dict(zip(Y_RANGE, range(len(Y_RANGE))))

In [16]:
def move2cord(move):
    if move == '--':
        return 15, 0
    
    move = (move[0], move[1:])
    x = X_MAPPING[move[1]]
    y = Y_MAPPING[move[0]]
    
    return x, y

In [18]:
def idx2cord(idx):
    return idx // 15, idx % 15

In [19]:
def refresh_board():
    board = np.dstack((
            np.zeros((15, 15, 2), dtype=np.int8),
            np.ones((15, 15, 1), dtype=np.int8)
        ))
        
    # board[x, y, 2] <- 0 if we made step (x, y)
    # --//--         <- 1/-1 if it is allowed to move
    # Value means which player are going to move
    step_mask = np.dstack((
        np.full((15, 15, 2), False),
        np.full((15, 15, 1), True)
    ))
    next_player = PLAYERS[0]
    
    return board, step_mask, next_player

In [29]:
board, step_mask, next_player = refresh_board()
moves_f = [lambda x: input(), lambda x: policyNN2.predict(x)[0]]
unmarch_move = [lambda x: move2cord(x), lambda x: idx2cord(x.argmax())]
i = 0
while True:
    if i % 2 == 0:
        print(board[:, :, 0] | board[:, :, 1])
    move = moves_f[i % 2](np.expand_dims(board, 0))
    x, y = unmarch_move[i % 2](move)

    board[step_mask] = next_player
                
    if x != 15:
        board[x, y, ((3-next_player) % 3) % 2] = next_player
        board[x, y, 2] = 0
        step_mask[x, y, 2] = False
            
    next_player = -next_player
    i += 1

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
h5
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  

KeyboardInterrupt: 

In [26]:
-1 | 1

-1

In [3]:
a = np.ones((15, 15))

In [4]:
a[10, 4] = a[3, 3] = a[11,  7] = 0

In [5]:
a

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.

In [6]:
a[a == 1] = -1

In [8]:
b = np.zeros((15, 15))

In [9]:
b[a == -1] = 1

In [10]:
b

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.

In [11]:
b[:, 1][2] = 0

In [12]:
b

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.